In [ ]:

import numpy as np
import pandas as pd
import sklearn
import os


import cv2
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.nn.functional as F

import random
!pip install random2
import random2
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
!pip install kneed
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


import glob
import seaborn as sns
import matplotlib.cm as cm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

!pip install kmeans-pytorch
from kmeans_pytorch import kmeans, kmeans_predict

import gc

import faiss

from PIL import Image

Image.LOAD_TRUNCATED_IMAGES = True

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
gc.collect()

In [ ]:
base_path= 'C:/Users/laura/Google Drive/WikiArt2/wikiart'
genres = os.listdir(base_path)
genres_f = []
for genre in genres:
        path = os.path.join(base_path, genre)
        length = len(os.listdir(path))
        if length > 1500:
          genres_f.append(genre)

In [ ]:
base_path= 'C:/Users/laura/Google Drive/WikiArt2/wikiart'
genres = os.listdir(base_path)
training_data = []

#print(len(training_data))
genres_f = []
for genre in genres:
        path = os.path.join(base_path, genre)
        length = len(os.listdir(path))
        if length > 1500:
          genres_f.append(genre)

for genre in genres_f:
        path = os.path.join(base_path, genre)
        class_num = genres_f.index(genre) #assigning each art category a classification based on it's index in the list
        selected = random.sample(os.listdir(path), 1000) 
        for img in selected:
            img_array = cv2.imread(os.path.join(path,img))
            #img_array = Image.open(os.path.join(path,img))
            if img_array is None:
                    print('Wrong path:', path)
            else:
                    img_array = Image.fromarray(img_array)
                    aug_im = transform(img_array)
                    #batch_image = aug_im.unsqueeze(dim=0)
                    #sobel_im = combined(aug_im)
                    training_data.append([aug_im, class_num])
                    


In [ ]:
torch.save(training_data, 'tensor2.pt')

## load saved data

In [ ]:
with torch.no_grad():
    training_data = torch.load('tensor.pt')

In [ ]:
random.shuffle(training_data)

In [ ]:
trainloader = torch.utils.data.DataLoader(training_data, batch_size=256, shuffle=False, num_workers=2)


In [ ]:
trainfeature, trainlabel = next(iter(trainloader))
print(f"Feature Batch Shape: {trainfeature.size()}")
print(f"Label Batch Shape: {trainlabel.size()}")

# Simple AlexNet with labels

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform = transforms.Compose([
    transforms.Resize((227, 227)),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
])

In [ ]:
gc.collect()

In [ ]:
base_path= 'C:/Users/laura/Google Drive/WikiArt2/wikiart'
genres = os.listdir(base_path)
training_data = []

#print(len(training_data))
genres_f = []
for genre in genres:
        path = os.path.join(base_path, genre)
        length = len(os.listdir(path))
        if length > 1500:
          genres_f.append(genre)

for genre in genres_f:
        path = os.path.join(base_path, genre)
        class_num = genres_f.index(genre) #assigning each art category a classification based on it's index in the list
        selected = random.sample(os.listdir(path), 1000) 
        for img in selected:
            img_array = cv2.imread(os.path.join(path,img))
            #img_array = Image.open(os.path.join(path,img))
            if img_array is None:
                    print('Wrong path:', path)
            else:
                    img_array = Image.fromarray(img_array)
                    aug_im = transform(img_array)
                    #batch_image = aug_im.unsqueeze(dim=0)
                    #sobel_im = combined(aug_im)
                    training_data.append([aug_im, class_num])

In [ ]:
#  Load images with ImageFolder
#base_path= 'C:/Users/laura/Google Drive/WikiArt2/wikiart'
#data = datasets.ImageFolder('base_path', transform=transform)


In [ ]:
trainloader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=False, num_workers=2)


In [ ]:

class AlexNet(nn.Module):
    def __init__(self, num_classes=14):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
num_classes = 14
num_epochs = 50
batch_size = 64
learning_rate = 0.005

model = AlexNet(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  


# Train the model
total_step = len(trainloader)

In [ ]:

total_step = len(trainloader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in trainloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total)) 

## whole process of AlexNet > Cluster > AlexNet


In [ ]:
gc.collect()

In [ ]:
# First alexnet

class AlexNet1(nn.Module):
    def __init__(self, num_classes=4096):
        super(AlexNet1, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
trainloader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=False, num_workers=2)


In [ ]:
#Parameters
num_classes = 4096
batch_size = 64
learning_rate = 0.005

model = AlexNet1(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  


# Train the model
total_step = len(trainloader)




total_step = len(trainloader)
num_epochs = 1
comb_scores = []

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):  
        # Move tensors to the configured device
        images = images.to(device)
        #labels = labels.to(device) # don't use labels
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        comb_scores.append(outputs.detach().cpu().numpy())
        
    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

comb_scores = np.concatenate(comb_scores)

print('Finished Training of AlexNet1')

In [ ]:
def RunPCA(comb_scores):
    mat = faiss.PCAMatrix(d_in=4096, d_out=256, eigen_power=-0.5)
    mat.train(comb_scores)
    x_pca = mat.apply_py(comb_scores)
    return x_pca

In [ ]:
X_PCA = RunPCA(comb_scores)

## L2 normalisation

In [ ]:
def RunL2(x_pca):
    norm = np.linalg.norm(x_pca, axis=1)
    x_l2 = x_pca / norm[:, np.newaxis]
    x_l2 = torch.tensor(x_l2)
    return x_l2

In [ ]:
X_L2 = RunL2(X_PCA)

## K-means

In [ ]:
gc.collect()

In [ ]:
def RunK_means(x_l2):
    # set device
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    
    # k-means
    cluster_ids_x, cluster_centers = kmeans(
        x_l2,
        num_clusters= 14, 
        distance='euclidean', 
        device=device
    )
    return cluster_ids_x


In [ ]:
CLUSTER_IDS_x  = RunK_means(X_L2)

In [ ]:
torch.bincount(CLUSTER_IDS_x)

In [ ]:
plt.hist(CLUSTER_IDS_x)
plt.title('cluster membership counts');

In [ ]:
gc.collect()

## Final Alexnet

In [ ]:
# Second alexnet

class AlexNet2(nn.Module):
    def __init__(self, num_classes=14):
        super(AlexNet2, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
X=[]
y= []

for features, label in training_data: 
    X.append(features)
    y.append(label)

In [ ]:
CLUSTER_IDS_x

In [ ]:
y

In [ ]:
trainloader = torch.utils.data.DataLoader([ [X[i], CLUSTER_IDS_x[i]] for i in range(len(training_data))], batch_size=64, shuffle=False, num_workers=2)


In [ ]:
trainloader.dataset

In [ ]:
num_classes = 14
batch_size = 64
learning_rate = 0.005

model = AlexNet2(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  


# Train the model
total_step = len(trainloader)
total_step = len(trainloader)
num_epochs = 50

for epoch in range(num_epochs):
    comb_scores2 = []
    label = []
    for i, (images, labels) in enumerate(trainloader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in trainloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            comb_scores2.append(predicted.detach().cpu().numpy())
            label.append(labels)
            del images, labels, outputs
    
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total)) 
    comb_scores2 = np.concatenate(comb_scores2)
    label = np.concatenate(label)

In [ ]:
with open('comb_scores2.npy', 'wb') as f:
    np.save(f, comb_scores2)


In [ ]:
with open('label.npy', 'wb') as f:
    np.save(f, label)

In [ ]:
with open('comb_scores2.npy', 'rb') as f:
    comb_scores2 = np.load(f)


In [ ]:
with open('label.npy', 'rb') as f:
    label = np.load(f)

In [ ]:
with open('y.npy', 'rb') as f:
    y = np.load(f)

In [ ]:
labels_numbers = []

def create_labels():
    for genre in genres_f:
        class_num = genres_f.index(genre) #assigning each art category a classification based on it's index in the list
        labels_numbers.append([genre, class_num])


create_labels()

In [ ]:
from sklearn.metrics import confusion_matrix

#Get the confusion matrix
cf_matrix = confusion_matrix(label, comb_scores2)

ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=False, 
            fmt='.2%', cmap='Blues')

ax.set_xlabel('\nClassified Style')
ax.set_ylabel('Pseudo label ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(labels_numbers, rotation = 90)
ax.yaxis.set_ticklabels(labels_numbers, rotation = 360)

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
# wrt ground truth labels

In [ ]:
# mapping labels from cluster to original labels
def get_reference_dict(clusters,data_label):
    reference_label = {}
    # For loop to run through each label of cluster label
    for i in range(len(np.unique(clusters))):
        index = np.where(clusters == i,1,0)
        num = np.bincount(data_label[index==1])
        if np.size(num) != 0:
            num = num.argmax()
            reference_label[i] = num
        else:
            reference_label[i] = 6
    return reference_label
# Mapping predictions to original labels
def get_labels(clusters,reference_labels):
    temp_labels = np.random.rand(len(clusters))
    for i in range(len(clusters)):
        temp_labels[i] = reference_labels[clusters[i]]
    return temp_labels

In [ ]:
reference_labels = get_reference_dict(comb_scores2, y[:13871])
predicted_labels = get_labels(comb_scores2,reference_labels)
print(f"Accuracy for k = 14: ", accuracy_score(predicted_labels,y[:13871]))

In [ ]:
len(y[:13871])

In [ ]:

#Get the confusion matrix
cf_matrix = confusion_matrix(y[:13871], predicted_labels)

In [ ]:
ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=False, 
            fmt='.2%', cmap='Blues')

ax.set_xlabel('\nPredicted Style')
ax.set_ylabel('Actual Style ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(labels_numbers, rotation = 90)
ax.yaxis.set_ticklabels(labels_numbers, rotation = 360)

## Display the visualization of the Confusion Matrix.
plt.show()

## k = 64?

In [ ]:
gc.collect()

In [ ]:
trainloader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=False, num_workers=2)


In [ ]:
def RunPCA(comb_scoresK):
    mat = faiss.PCAMatrix(d_in=4096, d_out=256, eigen_power=-0.5)
    mat.train(comb_scores)
    x_pca = mat.apply_py(comb_scoresK)
    return x_pca

In [ ]:
#Parameters
num_classes = 4096
batch_size = 64
learning_rate = 0.005

model = AlexNet1(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  


# Train the model
total_step = len(trainloader)

total_step = len(trainloader)
num_epochs = 1
comb_scoresK = []

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):  
        # Move tensors to the configured device
        images = images.to(device)
        #labels = labels.to(device) # don't use labels
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        comb_scoresK.append(outputs.detach().cpu().numpy())
        
    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

comb_scoresK = np.concatenate(comb_scoresK)

print('Finished Training of AlexNet1')

In [ ]:
X_PCA = RunPCA(comb_scoresK)

## L2 normalisation

In [ ]:
def RunL2(x_pca):
    norm = np.linalg.norm(x_pca, axis=1)
    x_l2 = x_pca / norm[:, np.newaxis]
    x_l2 = torch.tensor(x_l2)
    return x_l2

In [ ]:
X_L2 = RunL2(X_PCA)

## K-means

In [ ]:
gc.collect()

In [ ]:
def RunK_means(x_l2):
    # set device
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    
    # k-means
    cluster_ids_x, cluster_centers = kmeans(
        x_l2,
        num_clusters= 64, 
        distance='euclidean', 
        device=device
    )
    return cluster_ids_x


In [ ]:
CLUSTER_IDS_x  = RunK_means(X_L2)

In [ ]:
torch.bincount(CLUSTER_IDS_x)

In [ ]:
plt.hist(CLUSTER_IDS_x)
plt.title('cluster membership counts');

In [ ]:
gc.collect()

## Final Alexnet

In [ ]:
# Second alexnet

class AlexNet2(nn.Module):
    def __init__(self, num_classes=64):
        super(AlexNet2, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
X=[]
y= []

for features, label in training_data: 
    X.append(features)
    y.append(label)

In [ ]:
CLUSTER_IDS_x

In [ ]:
y

In [ ]:
trainloader = torch.utils.data.DataLoader([ [X[i], CLUSTER_IDS_x[i]] for i in range(len(training_data))], batch_size=64, shuffle=False, num_workers=2)


In [ ]:
trainloader.dataset

In [ ]:
num_classes = 64
batch_size = 64
learning_rate = 0.005

model = AlexNet2(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  


# Train the model
total_step = len(trainloader)
total_step = len(trainloader)
num_epochs = 50

for epoch in range(num_epochs):
    comb_scoresK = []
    labelK = []
    for i, (images, labels) in enumerate(trainloader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in trainloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            comb_scoresK.append(predicted.detach().cpu().numpy())
            labelK.append(labels)
            del images, labels, outputs
    
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total)) 
    comb_scoresK = np.concatenate(comb_scoresK)
    labelK = np.concatenate(labelK)

In [ ]:
with open('comb_scoresK.npy', 'wb') as f:
    np.save(f, comb_scoresK)


In [ ]:
with open('labelK.npy', 'wb') as f:
    np.save(f, labelK)

In [ ]:
with open('comb_scoresK.npy', 'rb') as f:
    comb_scoresK = np.load(f)

In [ ]:
with open('labelK.npy', 'rb') as f:
    labelK = np.load(f)

In [ ]:
predicted

In [ ]:
labelK

In [ ]:
# mapping labels from cluster to original labels
def get_reference_dict(clusters,data_label):
    reference_label = {}
    # For loop to run through each label of cluster label
    for i in range(64):
        index = np.where(clusters == i,1,0)
        num = np.bincount(data_label[index==1])
        if np.size(num) != 0:
            num = num.argmax()
            reference_label[i] = num
        else:
            reference_label[i] = 6
    return reference_label
# Mapping predictions to original labels
def get_labels(clusters,reference_labels):
    temp_labels = np.random.rand(len(clusters))
    for i in range(len(clusters)):
        temp_labels[i] = reference_labels[clusters[i]]
    return temp_labels

In [ ]:
len(np.unique(comb_scoresK))

In [ ]:
reference_labels = get_reference_dict(comb_scoresK,y[:13871])
predicted_labels = get_labels(comb_scoresK,reference_labels)
print(f"Accuracy for k = 64: ", accuracy_score(predicted_labels,y[:13871]))

In [ ]:
reference_labels

In [ ]:
from sklearn.metrics import confusion_matrix

#Get the confusion matrix
cf_matrix = confusion_matrix(labelK, comb_scoresK)

ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=False, 
            fmt='.2%', cmap='Blues')

ax.set_xlabel('\nPredicted label')
ax.set_ylabel('Pseudo label ');

## Ticket labels - List must be in alphabetical order
#ax.xaxis.set_ticklabels(labels_numbers, rotation = 90)
#ax.yaxis.set_ticklabels(labels_numbers, rotation = 360)

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
# wrt ground truth labels

In [ ]:
reference_labels = get_reference_dict(comb_scoresK, y[:13871])
predicted_labels = get_labels(comb_scoresK,reference_labels)
print(f"Accuracy for k = 64: ", accuracy_score(predicted_labels,y[:13871]))

In [ ]:

#Get the confusion matrix
cf_matrix = confusion_matrix(y[:13871], predicted_labels)

In [ ]:
ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=False, 
            fmt='.2%', cmap='Blues')

ax.set_xlabel('\nPredicted Style')
ax.set_ylabel('Actual Style ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(labels_numbers, rotation = 90)
ax.yaxis.set_ticklabels(labels_numbers, rotation = 360)

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
y

In [ ]:
import pandas as pd
results = pd.DataFrame([labelK, y]).T
results.columns = ['cluster', 'style']
results

match subclusters to ground truth styles. 

count style == subcluster >> 

New table: group per subcluster, filter max count per group, and keep subcluster and style.


In [ ]:
counted_results = results.groupby(["cluster", "style"]).size().reset_index(name="count")
filtered_results = counted_results.loc[counted_results.reset_index().groupby(['cluster'])['count'].idxmax()]
cluster_to_style = filtered_results[["cluster", "style"]].reset_index()

In [ ]:
# percentage of cluster in style
total = pd.DataFrame(counted_results.groupby(['cluster'])['count'].sum().reset_index())
#total
total.columns = ['cluster', 'total']
counted_results = counted_results.join(total, on='cluster', how = 'left', lsuffix='_left', rsuffix='_right')

In [ ]:
counted_results['%'] = counted_results['count']/counted_results['total']

In [ ]:
counted_results['max%'] = counted_results.groupby(['cluster_left'])['%'].transform(max)

In [ ]:
# Permanently changes the pandas settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
 
# All dataframes hereafter reflect these changes.
display(perc_of_cluster_style)
 




In [ ]:
filtered_results

Match predictions to styles through clusters

In [ ]:
predictions = pd.DataFrame(comb_scoresK)
predictions.columns = ['prediction']
predictions = predictions.join(cluster_to_style.set_index('cluster'), on='prediction')

In [ ]:
predictions['ground_truth'] = y

In [ ]:
(predictions['style'] == predictions['ground_truth']).sum() / len(predictions)

In [ ]:
counted_results[counted_results['%'] == counted_results['max%']]

In [ ]:
#reference_labels = get_reference_dict(cluster_to_style['cluster'], cluster_to_style['style'])
#predicted_labels = get_labels(cluster_to_style['cluster'],reference_labels)
from sklearn.metrics import accuracy_score

print(f"Accuracy for k = 14: ", accuracy_score(cluster_to_style['cluster'],cluster_to_style['style']))

In [ ]:
del training_data